<a href="https://colab.research.google.com/github/renatojmf/CM1-Software-Detection-Prediciton/blob/main/Extraction_MultiWOZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

In [43]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import pandas as pd
import numpy as np
import os
import time

print(tf.__version__)

2.5.0


In [53]:
dialogues = []
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_001.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_002.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_003.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_004.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_005.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_006.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_007.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_008.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_009.json'))
dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_010.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_011.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_012.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_013.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_014.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_015.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_016.json'))
# dialogues.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_017.json'))

dialogues_test = []
dialogues_test.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/test/dialogues_001.json'))
dialogues_test.append(pd.read_json('https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/test/dialogues_002.json'))

In [54]:
datas = []
for dialogues_ind in dialogues:
  for dialogue in dialogues_ind.turns:
    for sentences in dialogue:
      if sentences['speaker'] == 'USER' and sentences['frames'][0]['service'] == 'restaurant':
        datas.append(pd.Series({'Sentence': sentences['utterance'],'Intent': sentences['frames'][0]['state']['active_intent']}))

In [55]:
tests = []
for dialogues_ind in dialogues_test:

  for dialogue in dialogues_ind.turns:
    for sentences in dialogue:
      if sentences['speaker'] == 'USER' and sentences['frames'][0]['service'] == 'restaurant':
        tests.append(pd.Series({'Sentence': sentences['utterance'],'Intent': sentences['frames'][0]['state']['active_intent']}))

In [56]:
df = pd.DataFrame(datas)
df_test = pd.DataFrame(tests)

# Análises

In [51]:
labels = set()

def file2Examples(file_name):
  examples=[]
  example = [[],[]]
  f = file_name.values.tolist()

  for line in f:
    
    input_output_split = line
    print(input_output_split)
    
    if len(input_output_split)==4:
      example[0].append(input_output_split[0])
      example[1].append(input_output_split[-1])
      labels.add(input_output_split[-1])

    elif len(input_output_split)==0:
      examples.append(example)
      example=[[],[]]
    else:
      example=[[],[]]

    
  return examples

In [ ]:
train_examples = file2Examples(df)
test_examples  = file2Examples(df_test)
print(train_examples)

# Spacy

In [ ]:
! pip install -U spacy
! python -m spacy download en

In [45]:
# for natural language processing: named entity recognition
import spacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
# for visualizations
%matplotlib inline

In [57]:
tokens = nlp(''.join(str(df.Sentence.tolist())))

In [58]:
items = [x.text for x in tokens.ents]
Counter(items).most_common(20)

[('today', 291),
 ('Cambridge', 272),
 ('4', 231),
 ('3', 226),
 ('8', 207),
 ('Indian', 199),
 ('Sunday', 194),
 ('Saturday', 193),
 ('7', 188),
 ('2', 187),
 ('Friday', 187),
 ('6', 183),
 ('Italian', 180),
 ('Wednesday', 180),
 ('Chinese', 173),
 ('5', 172),
 ('Monday', 169),
 ('Tuesday', 159),
 ('Thursday', 155),
 ('1', 125)]

In [59]:
norp_list = []
for ent in tokens.ents:
    if ent.label_ == 'NORP':
        norp_list.append(ent.text)
        
norp_counts = Counter(norp_list).most_common(20)
df_norp = pd.DataFrame(norp_counts, columns =['text', 'count'])
df_norp.head()

,text,count
0,Indian,199
1,Italian,180
2,Chinese,173
3,chinese,118
4,British,114
